In [2]:
import torch
import numpy as np


In [8]:
edges = torch.tensor(([[[0,1,0,0],
                        [1,0,1,0],
                        [0,1,0,1],
                        [0,0,1,0]]]))
nodes = torch.tensor(([[[0,0,0,1,1],
                        [0,0,1,1,0],
                        [0,1,0,0,1],
                        [1,0,0,0,1
                        ]]]))
edges.shape,nodes.shape
edges = edges.view(1,4,4,1)
#single graph with multiple nodes and edges

In [16]:
adjacency.nonzero(as_tuple=True),adjacency.sum(-1).nonzero(as_tuple=True)

((tensor([0, 0, 0, 0, 0, 0]),
  tensor([0, 1, 1, 2, 2, 3]),
  tensor([1, 0, 2, 1, 3, 2])),
 (tensor([0, 0, 0, 0]), tensor([0, 1, 2, 3])))

In [9]:
adjacency = torch.sum(edges, dim=3)
print("adj is ",adjacency.shape)
(edge_batch_batch_idc,
edge_batch_node_idc,
edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
(node_batch_batch_idc,
node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)
print("nodes various ares",(node_batch_batch_idc,node_batch_node_idc))

adj is  torch.Size([1, 4, 4])
nodes various ares (tensor([0, 0, 0, 0]), tensor([0, 1, 2, 3]))


In [53]:
same_batch = node_batch_batch_idc.view(-1,1)== edge_batch_batch_idc
same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc
message_summation_matrix = (same_batch * same_node).float()
same_node,same_batch

(tensor([[ True, False, False, False, False, False],
         [False,  True,  True, False, False, False],
         [False, False, False,  True,  True, False],
         [False, False, False, False, False,  True]]),
 tensor([[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]]))

In [12]:
edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]
print("edge_batch_edges ",edge_batch_edges.shape,'hello ',edges.shape)
hidden_node_features=100
hidden_nodes = torch.zeros(nodes.shape[0],
                            nodes.shape[1],
                            hidden_node_features,
                            device='cuda')
hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()#padding upto 13 nodes,with features to 100....we have only 9 features here
node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]#picking out the same batch


edge_batch_edges  torch.Size([6, 1]) hello  torch.Size([1, 4, 4, 1])


In [75]:
nodes.shape

torch.Size([1, 4, 5])

In [14]:
message_passes = 4
for _ in range(message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]#getting hi  13*13*100

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]#getting neighs(hj)  13*13*1#why its 1
            print("in message pass ",edge_batch_nghbs.shape, "nodes ",edge_batch_nodes.shape)
            break
            message_terms    = message_terms(edge_batch_nodes,  
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

in message pass  torch.Size([6, 100]) nodes  torch.Size([6, 100])


In [69]:
edge_batch_edges.shape

torch.Size([6, 1])

In [68]:
edge_batch_edges = edge_batch_edges.view(6,1)#and in bond case it will be something like.... 6,4,1

In [71]:
edges_v               = edge_batch_edges.view(-1,1, 1)
node_neighbours_v     = edges_v * edge_batch_nghbs.view(-1,1,100).cpu()
node_neighbours_v.shape
outs = []
for i in range(1):
    outs.append(edge_batch_edges[:,i,:]*msg_nns[i](node_neighbours_v[:,i:,])) #here we upper part we pull the edges type bw i,j. it should be a matrix like [1,4] but in our case we use single matrix only
output = sum(outs)
messages = torch.matmul(message_summation_matrix, output)
print(messages.shape)

RuntimeError: The size of tensor a (6) must match the size of tensor b (100) at non-singleton dimension 2

In [77]:
def s2v_message_passing(nodes,neighs,edges):
    edges_v = edges.view(-1,1,1)#it gives value for edges.....type of edge..size->(6,4)->in molecule with 6 edges kind
    neighs = edges_v*neighs.view(-1,1,1)#multiplying each by this number
    outs = []
    for i in range(1):
        outs.append(nodes[:,i,:]*msg_nns[i](edges_v[:,i,:]))
    output = sum(outs)
    return sum


In [66]:
node_batch_nodes.shape,outs[0].shape,edge_batch_nghbs.shape

(torch.Size([4, 100]), torch.Size([6, 6, 100]), torch.Size([6, 100]))

In [60]:
node_batch_nodes[:messages.shape[0],:messages.shape[1],:] = messages


IndexError: too many indices for tensor of dimension 2

In [58]:
message_summation_matrix.shape,output.shape

(torch.Size([4, 6]), torch.Size([6, 6, 100]))

In [56]:
messages = torch.matmul(message_summation_matrix, output)
print(messages.shape)

torch.Size([6, 4, 100])


In [50]:
output.shape

torch.Size([6, 6, 100])

In [51]:
edge_batch_nghbs.shape

torch.Size([6, 100])

In [22]:
edge_batch_nghbs.view(-1,1,100).cpu().shape

torch.Size([6, 1, 100])

In [36]:

msg_nns = []
for _ in range(4):#number of edge features
            msg_nns.append(
                MLP(
                    in_features=100,
                    hidden_layer_sizes=[250] * 4,
                    out_features=100,#message size
                    dropout_p=0,
                )
            )


In [6]:
import torch
from torch import nn
from collections import namedtuple

In [8]:

class MLP(torch.nn.Module):
    def __init__(self, in_features : int, hidden_layer_sizes : list, out_features : int,
                 dropout_p : float) -> None:
        super().__init__()

        activation_function = torch.nn.SELU
        # create list of all layer feature sizes
        fs = [in_features, *hidden_layer_sizes, out_features]
        # create list of linear_blocks
        layers = [self._linear_block(in_f, out_f,
                                     activation_function,
                                     dropout_p)
                  for in_f, out_f in zip(fs, fs[1:])]
        # concatenate modules in all sequentials in layers list
        layers = [module for sq in layers for module in sq.children()]

        # add modules to sequential container
        self.seq = torch.nn.Sequential(*layers)

    def _linear_block(self, in_f : int, out_f : int, activation : torch.nn.Module,
                      dropout_p : float) -> torch.nn.Sequential:
        
        # bias must be used in most MLPs in our models to learn from empty graphs
        linear = torch.nn.Linear(in_f, out_f, bias=True)
        torch.nn.init.xavier_uniform_(linear.weight)
        return torch.nn.Sequential(linear, activation(), torch.nn.AlphaDropout(dropout_p))

    def forward(self, layers_input : torch.nn.Sequential) -> torch.nn.Sequential:
        return self.seq(layers_input)


In [17]:
hyperparameters = {
        "enn_depth"            : 4,
        "enn_dropout_p"        : 0.0,
        "enn_hidden_dim"       : 250,
        "mlp1_depth"           : 4,
        "mlp1_dropout_p"       : 0.0,
        "mlp1_hidden_dim"      : 500,
        "mlp2_depth"           : 4,
        "mlp2_dropout_p"       : 0.0,
        "mlp2_hidden_dim"      : 500,
        "gather_att_depth"     : 4,
        "gather_att_dropout_p" : 0.0,
        "gather_att_hidden_dim": 250,
        "gather_emb_depth"     : 4,
        "gather_emb_dropout_p" : 0.0,
        "gather_emb_hidden_dim": 250,
        "gather_width"         : 100,
        "hidden_node_features" : 100,
        "message_passes"       : 3,
        "message_size"         : 100,
    }
import json
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
constants = dotdict(hyperparameters)


In [10]:
from ashishcode import load_molecule,MolecularGraph,params
path = "data\pre-training\gdb13_1K\Train.smi"
molecule_set = load_molecule(path)
print(molecule_set)
for mol in molecule_set:
    molecule = MolecularGraph(mol)
    print(molecule)
    break


True


In [51]:

class SummationMPNN(torch.nn.Module):
    """
    Abstract `SummationMPNN` class. Specific models using this class are
    defined in `mpnn.py`; these are MNN, S2V, and GGNN.
    """
    def __init__(self, constants : namedtuple):

        super().__init__()

        self.hidden_node_features = constants.hidden_node_features
        self.edge_features        = constants.n_edge_features
        self.message_size         = constants.message_size
        self.message_passes       = constants.message_passes
        self.constants            = constants

    def forward(self, nodes : torch.Tensor, edges : torch.Tensor) -> None:
        adjacency = torch.sum(edges, dim=3)
        # **note: "idc" == "indices", "nghb{s}" == "neighbour(s)"
        (edge_batch_batch_idc,
         edge_batch_node_idc,
         edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
        #print("sizes are edge_batch_node_idc ",edge_batch_node_idc)

        (node_batch_batch_idc, node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)

        same_batch = node_batch_batch_idc.view(-1, 1) == edge_batch_batch_idc
        same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc

        # element ij of `message_summation_matrix` is 1 if `edge_batch_edges[j]`
        # is connected with `node_batch_nodes[i]`, else 0
        message_summation_matrix = (same_batch * same_node).float()

        edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]

        # pad up the hidden nodes
        hidden_nodes = torch.zeros(nodes.shape[0],
                                   nodes.shape[1],
                                   self.hidden_node_features,
                                   device=self.constants.device)
        hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()
        hidden_nodes_original = hidden_nodes.clone()
        node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]
        

        for _ in range(self.message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]

            #print("hello ji ",edge_batch_nghbs.shape,edge_batch_nodes.shape,hidden_nodes.shape)

            message_terms = self.message_terms(edge_batch_nodes,
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

            if len(message_terms.size()) == 1:  # if a single graph in batch
                message_terms = message_terms.unsqueeze(0)

            # the summation in eq. 1 of the NMPQC paper happens here
            messages = torch.matmul(message_summation_matrix, message_terms)

            node_batch_nodes = self.update(node_batch_nodes, messages)
            hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :] = node_batch_nodes.clone() #updated the hidden states
        node_mask = adjacency.sum(-1) != 0
        output    = self.readout(hidden_nodes, hidden_nodes_original, node_mask)
        return output


In [61]:
# from graphinvent.gnn.modules import GlobalReadout 
class GGNN(SummationMPNN):
    def __init__(self,constants) -> None:
        super().__init__(constants)
        self.constants       = constants
        message_weights      = torch.Tensor(self.constants.message_size,
                                            self.constants.hidden_node_features,
                                            4)#edge features
        if False:#"cuda" == "cuda":
            message_weights = message_weights.to("cuda", non_blocking=True)
        
    
        self.message_weights = torch.nn.Parameter(message_weights)

        self.gru             = torch.nn.GRUCell(
            input_size=self.constants.message_size,
            hidden_size=self.constants.hidden_node_features,
            bias=True
        )
        
        self.msg_nns = []
        for _ in range(4):#number of edge features
                    self.msg_nns.append(
                        MLP(
                            in_features=100,
                            hidden_layer_sizes=[250] * 4,
                            out_features=100,#message size
                            dropout_p=0,
                        )
                    )

        self.gather = GraphGather(node_features=100, hidden_node_features=100, out_features=100, 
                                    att_depth = constants.gather_att_depth,
                                    att_hidden_dim=constants.gather_att_hidden_dim,
                                    att_dropout_p=0.4,
                                    emb_depth=constants.gather_emb_depth, emb_hidden_dim=constants.gather_emb_hidden_dim,
                                  emb_dropout_p=0.0, big_positive=0.1)
        
        self.APDReadout = GlobalReadout()

        self.reset_parameters()

    def reset_parameters(self) -> None:
        import math
        stdev = 1.0 / math.sqrt(self.message_weights.size(1))
        self.message_weights.data.uniform_(-stdev, stdev)

    def message_terms(self, nodes : torch.Tensor, node_neighbours : torch.Tensor,
                        edges : torch.Tensor) -> torch.Tensor:
        
        edges_v = edges.view(-1,4,1)#it gives value for edges.....type of edge..size->(6,4)->in molecule with 6 edges kind
        outs = []
        for i in range(4):
            # print("in last loop ",edges_v[:,i,:].shape,node_neighbours.shape)
            outs.append(edges_v[:,i,:]*self.msg_nns[i](node_neighbours))#check node neigh element wise help
        output = sum(outs)
        return output

        
    #torch.broadcast_to(x, (3, 3))
    def update(self, nodes : torch.Tensor, messages : torch.Tensor) -> torch.Tensor:
        return self.gru(messages, nodes)

    def readout(self, hidden_nodes : torch.Tensor, input_nodes : torch.Tensor,
                node_mask : torch.Tensor) -> torch.Tensor:
        # graph_embeddings = torch.sum(hidden_nodes, dim=1)
        # print("input to network ",hidden_nodes.shape,input_nodes.shape)
        output_gather =  self.gather(input_nodes,hidden_nodes,node_mask)
        output        = self.APDReadout(hidden_nodes,output_gather)
        return output



In [65]:
network = GGNN(constants)
node,edge = molecule.get_graph_state()
nodes,edges = torch.Tensor(node).view((1,10,9)),torch.Tensor(edge).view((1,10,10,4))
print("output of final net is ,",network(nodes,edges).shape)


in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
in last loop  torch.Size([24, 1]) torch.Size([24, 100])
input to network  torch.Size([1, 10, 100]) torch.Size([1, 10, 100])
shape of feat in global readout is  torch.Size([1, 10, 100])
graph embedding shapes  torch.Size([1, 100])
api is  torch.Size([1, 1, 100])
dims fadd ip torch.Size([1, 10, 100]) torch.Size([1, 1, 100]) torch.Size([1, 11, 100])
final shape  torch.Size([1, 23, 100

In [60]:

class GraphGather(torch.nn.Module):
    """
    GGNN readout function.
    """
    def __init__(self, node_features : int, hidden_node_features : int,
                 out_features : int, att_depth : int, att_hidden_dim : int,
                 att_dropout_p : float, emb_depth : int, emb_hidden_dim : int,
                 emb_dropout_p : float, big_positive : float) -> None:

        super().__init__()

        self.big_positive = big_positive

        self.att_nn = MLP(
            in_features=node_features + hidden_node_features,
            hidden_layer_sizes=[att_hidden_dim] * att_depth,
            out_features=out_features,
            dropout_p=att_dropout_p
        )

        self.emb_nn = MLP(
            in_features=hidden_node_features,
            hidden_layer_sizes=[emb_hidden_dim] * emb_depth,
            out_features=out_features,
            dropout_p=emb_dropout_p
        )

    def forward(self, hidden_nodes : torch.Tensor, input_nodes : torch.Tensor,
                node_mask : torch.Tensor) -> torch.Tensor:
        """
        Defines forward pass.
        """
        Softmax     = torch.nn.Softmax(dim=1)

        cat         = torch.cat((hidden_nodes, input_nodes), dim=2)
        #energy_mask = (node_mask == 0).float() * self.big_positive
        #energies    = self.att_nn(cat) - energy_mask.unsqueeze(-1)
        #attention   = Softmax(energies)
        attention = self.att_nn(cat)#activation is SELU-->done automatically inside
        embedding   = self.emb_nn(hidden_nodes)#this is mlp-a-->

        return torch.sum(attention * embedding, dim=1)
class GlobalReadout(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp1 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlp2 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        
        self.mlp3 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlp4 = MLP(in_features=2*constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlpt = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
          
    def forward(self,features,g):
        print("shape of feat in global readout is ",features.shape)
        #g= torch.sum(features,dim=1)
        # print("graph embedding shapes ",g.shape)
        g = g.view(1,1,100)
        #g = torch.broadcast_to(g, (1,10, 100))
        # print("api is ",g.shape)
        fadd1 = self.mlp1(features)
        fconn1 = self.mlp2(features)  

        # print("dims fadd ip",fconn1.shape,g.shape,torch.cat([fadd1,g],dim=1).shape)
        fadd = self.mlp3(torch.cat([fadd1,g],dim=1)).unsqueeze(dim=1)
        fconn = self.mlp3(torch.cat([fconn1,g],dim=1)).unsqueeze(dim=1)

        fterm = self.mlpt(g)
        cat = torch.cat((fadd.squeeze(dim=1), fconn.squeeze(dim=1), fterm), dim=1)
        # print("final shape ",cat.shape)
        return cat
        #apd = self.Softmax()....from original code its removed

    
            